# Table of Contents
## 1. [Data-Preprocessing](#Data-Preprocessing)
## 2. [Main Script](#Main-script)

# `src/preprocessing-utils.R` $\rightarrow$ `src/preprocessing_utils.py`

In [26]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [390]:
def as_quarter(months):
    """
    월(month) 값을 분기(quarter) 값으로 전환
    Example: (1, 2, 3, 4, 5, 6) -> (1, 1, 1, 2, 2, 2)
    
    INPUT:
      x: 1 ~ 12 사이의 integer vector
    RETURN:
      월 -> 분기로 전환된 integer vector
    """
    months = pd.to_numeric(months)
    if not np.all(np.isin(months, np.arange(1, 13))):
        raise ValueError("range of months exceeds 1~12")
    return (np.array(months) - 1) // 3 + 1

In [273]:
def is_quarter_interval(date_str):
    """
    시간 변수가 날짜 형식(ex: "20010131")인지 
    혹은 분기 형식(ex: "2001/1 Quarter)인지 검사
    
    INPUT:
      x: date string vector
    RETURN:
      날짜 형식이면 FALSE, 분기 형식이면 TRUE
    """
    return len(str(date_str)) == 5

In [413]:
def reshape_long(df):
    """
    Short form 데이터를 long form 데이터로 변환
    시간 변수가 날짜 형식인 경우 분기 형식으로 변환
    
    INPUT:
      data: raw data를 불러들인 data frame
    RETURN:
      long form으로 전환된 data frame
    """
    df = gather(df)
    df.val = pd.to_numeric(df.val)
    df = tidyup_timeframe(df, is_quarter_interval(df.time[0]))
    return df

In [265]:
def gather(df):
    df.rename(index=str, columns={"Unnamed: 1": "code", "Unnamed: 2": "name"}, inplace=True)
    df.columns = ["y"+format_quarter(str(c)) if c[0].isnumeric() else c for c in df.columns]
    df = pd.wide_to_long(df, stubnames="y", i=["code", "name"], j="time")
    df = pd.DataFrame(df.to_records()).rename(index=str, columns={"y": "val"})
    return df

In [266]:
def format_quarter(string):
    return string.replace(" ", "").replace("/", "").replace("Quarter", "").replace("SemiAnnual", "2").replace("Annual", "4")

In [409]:
def tidyup_timeframe(df, is_quarter):
    df.val = pd.to_numeric(df.val)
    if is_quarter:
        year_quarter = df.time.astype(str).str.extract('(.{4,4})(.{1,1})')
        year_quarter.columns = ["year", "quarter"]
        yq = year_quarter.year + "-" + year_quarter.quarter
        yq.name = "time"
        df = pd.concat([df.iloc[:, :2], yq, df.val], axis=1)
    else:
        year_quarter = df.time.astype(str).str.extract('(.{4,4})(.{2,2})')
        year_quarter.columns = ["year", "quarter"]
        yq = year_quarter.year + "-" + as_quarter(year_quarter.quarter).astype(str)
        yq.name = "time"
        df = pd.concat([df.iloc[:, :2], yq, df.val], axis=1)
        df = pd.DataFrame(df.groupby(["code", "name", "time"]).mean().to_records())
    df.sort_values(by=["code", "time"], inplace=True)
    return df

In [501]:
def preprocess(path, file_names, var_names, extension=".xls"):
    dfs = []
    for name in file_names:
        print(name, end=", ")
        file_path = os.path.join(path, name+extension)
        data = reshape_long( pd.read_excel(file_path, skiprows=5).iloc[1:, 1:] )
        dfs.append(data)

    vals = reduce(lambda x, y: x.merge(y, how="left", on=["code", "name", "time"]), dfs).iloc[:, 3:]
    vals.columns = var_names
    
    features = extract_features(vals)
    df = pd.concat([data.loc[:, ["code", "time"]], features], axis=1)
    return df

In [502]:
def extract_features(vals):
    leverage = vals.leverage
    asset_growth = vals.asset_growth
    shares_turnover = vals.trade_amount / vals.stock_num
    roa = vals.net_profit / vals.asset
    roe = vals.net_profit / vals.equity
    size = vals.market_cap
    pcr = vals.pcr
    per = vals.per
    equity_turnover = vals.equity_turnover
    volatility = vals.volatility
    logret = np.log(vals.price).diff()
    
    features = pd.concat([leverage, asset_growth, shares_turnover, roa, roe, size, 
                          pcr, per, equity_turnover, volatility, logret], axis=1)
    features.columns = ["leverage", "asset_growth", "shares_turnover", "roa", "roe", "size", 
                        "pcr", "per", "equity_turnover", "volatility", "logret"]
    return features

---

# Data Preprocessing
## `scripts/01_data-preprocessing.R` $\rightarrow$ `scripts/01_data-preprocessing.py`

In [1]:
from src.preprocessing_utils import *

## Stock data

In [2]:
path = os.path.join("data", "raw")
file_names = ["asset", "asset-growth", "equity", "equity-turnover",
              "leverage", "market-cap", "net-profit", "pcr", "per",
              "stock-number", "stock-price", "trade-amount", "volatility"]
var_names = ["asset", "asset_growth", "equity", "equity_turnover",
             "leverage", "market_cap", "net_profit", "pcr", "per",
             "stock_num", "price", "trade_amount", "volatility"]
extension = ".xls"

In [3]:
stock_tbl = preprocess(path, file_names, var_names, extension=".xls")
stock_tbl.head()

asset, asset-growth, equity, equity-turnover, leverage, market-cap, net-profit, pcr, per, stock-number, stock-price, trade-amount, volatility, 

,code,time,leverage,asset_growth,shares_turnover,roa,roe,size,pcr,per,equity_turnover,volatility,logret
0,000020,1997-1,NaN,NaN,0.006314,NaN,NaN,8.850400e+10,NaN,15.586667,NaN,60.255281,NaN
1,000020,1997-2,206.92,23.60,0.015739,0.002507,0.007696,1.104880e+11,NaN,11.053333,2.22,62.004616,0.235505
2,000020,1997-3,NaN,NaN,0.004035,NaN,NaN,1.087867e+11,NaN,10.543333,NaN,56.334550,-0.040738
3,000020,1997-4,324.23,12.41,0.004633,0.009214,0.039088,4.613333e+10,NaN,4.346667,2.89,62.702979,-0.847219
4,000020,1998-1,NaN,NaN,0.012191,NaN,NaN,5.973333e+10,NaN,4.750000,NaN,73.726084,0.120535


## KOSPI index

In [4]:
def unpack_df(df):
    df = df.reset_index()
    df["time"] = df[['time', 'level_1']].astype(str).apply(lambda x: '-'.join(x), axis=1)
    df = pd.concat([df.time, df.price], axis=1)
    df.columns = ["time", "logret"]
    return df

In [14]:
kospi = pd.read_excel("data/raw/kospi-index.xlsx", names=["time", "price"])
g = kospi.groupby([pd.DatetimeIndex(kospi.time).year, as_quarter(pd.DatetimeIndex(kospi.time).month)])
g = np.log(g.mean()).diff()
kospi = unpack_df(g)
kospi.head()

,time,logret
0,1997-1,NaN
1,1997-2,0.082477
2,1997-3,-0.064037
3,1997-4,-0.499768
4,1998-1,0.247514


## Risk-free rate

In [10]:
risk_free = pd.read_excel("data/raw/cd-risk-free.xlsx", names=["time", "r"])
risk_free.time = risk_free.time.str.replace("/", "-").str.split().str[0]
risk_free.r = np.log(1 + risk_free.r / 100)
risk_free.head()

,time,r
0,1997-1,0.119707
1,1997-2,0.118287
2,1997-3,0.119115
3,1997-4,0.145110
4,1998-1,0.204599


## Save processed data

In [11]:
stock_tbl.to_csv("data/processed/stock2.csv", index=False)
kospi.to_csv("data/processed/kospi2.csv", index=False)
risk_free.to_csv("data/processed/risk_free2.csv", index=False)

---

# Portfolio*
## `src/functions-portfolio.R`

0.04658314275080883

In [ ]:
def get_weight(x, method, risk_free=None):
    n_cluster = x.shape[1]
    zeros = np.zeros(n_cluster).T
    
    if method == "GMV":
        a = np.c_[ np.ones(n_cluster), np.diag(np.ones(n_cluster)) ]
        b = np.array([1] + [0]*n_cluster)
    elif method == "Tangency":
        if not risk_free:
            raise ValueError("method is 'Tangency'. 'risk_free' should not be None")
        rf = risk_free.r.mean()
        a = 
        b = 

In [26]:
n_cluster = 3

In [27]:
np.c_[ np.ones(3), np.diag(np.ones(n_cluster)) ]

array([[1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 1.]])

In [28]:
np.array([1] + [0]*n_cluster)

array([1, 0, 0, 0])

---

# Clustering**
## `src/functions-clustering.R`

In [5]:
def time_slice(data, time_idx):
    as_list = list(time_idx)
    return stock_tbl.groupby(["code"]).nth(as_list).reset_index()

In [57]:
def time_expand(data, skip=[0,1]):
    cols = [col for col in range(data.shape[1]) if col not in skip]
    
    while len(set(data.time)) > 1:
        lagged = data.iloc[:, cols].shift(1)
        lagged.columns = [f"x{c}" for c in cols]
        
        data = pd.concat([data, lagged], axis=1)
        data = data.groupby(["code"], as_index=False).apply(lambda x: x.iloc[1:]).reset_index(drop=True)
        
        cols = np.array(cols) + len(cols)
        
    return data

In [71]:
def scale_tbl(data, skip=[0,1]):
    idx = [i for i in range(data.shape[1]) if i not in skip]
    vals = data.iloc[:, idx]
    data.iloc[:, idx] = (vals - vals.mean()) / vals.std()
    return data.reset_index(drop=True)

### **PCA에서 데이터 절반이 날아감**

In [79]:
from sklearn.decomposition import PCA

In [148]:
def pca(data, skip=[0,1], threshold=.8):
    """
    데이터에 PCA(주성분분석)을 수행한다.
    
    INPUT:
      data: data frame
      skip: PCA 대상에서 제외할 열 번호 (integer vector)
      threshold: 주성분 개수 선택의 기준이 되는 변동의 비율 (0 ~ 1)
    
    RETURN:
      변수들이 주성분으로 대체된 data frame
    """
    idx = [i for i in range(data.shape[1]) if i not in skip]
    omit_na = data.iloc[:, idx].dropna()    #### 여기 dropna() 때문에 데이터 절반이 날아감
    
    pca = PCA()
    x_pc = pd.DataFrame(pca.fit_transform(omit_na))
    n_pc = np.where(pca.explained_variance_ratio_.cumsum() > threshold)[0][0] + 1
    x_pc = x_pc.iloc[:, :n_pc]
    x_pc.columns = [f"PC{c+1}" for c in x_pc.columns]
    
    df_pc = pd.concat([data.dropna().iloc[:, skip].reset_index(drop=True), x_pc], axis=1)
    return df_pc

In [ ]:
def add_factors_residual(data, risk_free):
    pass

In [ ]:
def add_market_residual(data, market, risk_free):
    pass

In [ ]:
def kmeanspp(x, k, algorithm, iter_max=500, nstart=20, trace=False):
    pass

In [ ]:
def get_kmeans_tbl(data, ncmin=2, ncmax=5):
    pass

In [ ]:
def kmeans_with(data, wth, market, risk_free):
    pass

In [ ]:
def integrate_return(rturn, weight):
    pass

In [ ]:
def get_cluster_return(data, time_idx, wth, market, risk_free):
    pass

In [149]:
pca(scale_tbl(stock_tbl))

,code,time,PC1,PC2,PC3,PC4
0,20,1999-2,-0.272279,-0.117506,0.115006,1.782137
1,20,1999-4,-0.252239,-0.185067,0.004330,0.480523
2,20,2000-1,-0.260210,-0.154559,0.049327,0.989953
3,20,2001-2,-0.309955,-0.101218,0.153028,2.736433
4,20,2001-3,-0.255205,-0.173066,-0.001545,0.377542
5,20,2001-4,-0.238938,-0.156821,0.004778,0.159197
6,20,2002-1,-0.257750,-0.140623,0.043302,0.852265
7,20,2002-2,-0.237482,-0.191993,-0.039112,-0.261369
8,20,2002-3,-0.237780,-0.179744,-0.018940,-0.081842
9,20,2002-4,-0.235098,-0.183559,-0.026679,-0.208802


In [69]:
vals.mean(), vals.std()

(leverage          265.575000
 asset_growth       18.005000
 sharesturnover      0.008582
 dtype: float64, leverage          82.950697
 asset_growth       7.912525
 sharesturnover     0.005140
 dtype: float64)

In [60]:
data = stock_tbl.copy().head().iloc[:, :5]
data.shape

(5, 5)

In [62]:
time_expand(data)

,code,time,leverage,asset_growth,sharesturnover,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
0,20,1998-1,NaN,NaN,0.012191,324.23,12.41,0.004633,NaN,NaN,0.004035,206.92,23.6,0.015739,NaN,NaN,0.006314


In [42]:
skip=[0,1]
cols = [col for col in range(data.shape[1]) if col not in skip]

In [43]:
lagged = data.iloc[:, cols].shift(1)
lagged.columns = [f"x{c}" for c in cols]

In [44]:
data = pd.concat([data, lagged], axis=1)
data.shape

(5, 24)

In [53]:
len(set(stock_tbl.time))

84

In [55]:
stock_tbl.groupby(["code"]).nth([1,2,3]).reset_index()

,code,time,leverage,asset_growth,sharesturnover,roa,roe,size,pcr,per,equity_turnover,volatility,logret
0,20,1997-2,206.92,23.60,0.015739,0.002507,0.007696,1.104880e+11,NaN,11.053333,2.22,62.004616,0.235505
1,20,1997-3,NaN,NaN,0.004035,NaN,NaN,1.087867e+11,NaN,10.543333,NaN,56.334550,-0.040738
2,20,1997-4,324.23,12.41,0.004633,0.009214,0.039088,4.613333e+10,NaN,4.346667,2.89,62.702979,-0.847219
3,30,1997-2,1949.03,4.97,0.002778,0.001302,0.026683,7.922000e+11,NaN,NaN,1.68,NaN,-0.048177
4,30,1997-3,NaN,NaN,0.003447,NaN,NaN,6.885000e+11,NaN,NaN,NaN,NaN,-0.140299
5,30,1997-4,2703.43,20.00,0.005399,-0.004181,-0.117222,5.054667e+11,NaN,NaN,2.25,NaN,-0.309033
6,40,1997-2,164.67,23.16,0.013669,0.005143,0.013613,7.108434e+10,0.636667,2.110000,2.24,NaN,-0.104141
7,40,1997-3,NaN,NaN,0.002400,NaN,NaN,9.455907e+10,0.513333,1.690000,NaN,NaN,-0.220858
8,40,1997-4,352.70,8.83,0.002777,-0.191695,-0.867803,3.483945e+10,0.190000,0.623333,2.60,NaN,-0.988159
9,50,1997-2,322.06,-0.83,0.000484,0.009460,0.039928,1.010284e+11,NaN,NaN,3.33,NaN,0.152189


---

# Main script

## `02_main.R`

In [ ]:
from src.clustering import *
from src.portfolio import *

In [15]:
import pandas as pd
import numpy as np

## Load data

In [4]:
stock_tbl = pd.read_csv("data/processed/stock.csv")
kospi = pd.read_csv("data/processed/kospi.csv")
risk_free = pd.read_csv("data/processed/risk_free.csv")

## Models

In [19]:
with_list = ["return", "market_residual", "factors", "factors_residual"]
n_time_list = [6, 8, 10, 12]
method_list = ["GMV", "Tangency"]

In [ ]:
start_list <- str_c(c("2002", "2005", "2008", "2011"), "-4")
end_list <- str_c(c("2005", "2008", "2011", "2014"), "-3")
valid_res_list <- list()

In [ ]:
start_list = ["2002-4", "2005-4", "2008-4", "2011-4"]
end_list = ["2005-3", "2008-3", "2011-3", "2014-3"]

validation_result = []
for i in range(4):
    st = start_list[i]
    en = end_list[i]
    valid_res_list[[i]] = evaluate_portfolio(stock_tbl, kospi, risk_free, start, end,
                                             with_list, n_time_list, method_list)